<center style = "font-family: 'Lucida Console', 'Courier New', monospace;">
    <img src = "https://blogger.googleusercontent.com/img/a/AVvXsEj6-rQw5r22Bt47BUTtW5bn_dcWT7zMeADwtvsAHS3kBt6w8eWTmCM649ZcJcvosIMup6flKFIaI8p4M9ZzH1yXpEaMRjvwwfVZ_hMqgXCxtwNzEK25vTa-J2ly20by3M1zx7rTymo-tBI6Fq-mj1SJfCOXsOz0Ou1Esi4h2omvQSW98AjsONsVS-EA" width=400 height = 200>
    <h1 style = "background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%);border-radius: 20px; font-size:30px">Tensorflow - Help Protect the Great Barrier Reef 🌟🐠</h1>
</center>

<div style = "background: rgb(224,224,224);border-radius: 42px;">
    <h1 style = "font-family: Consolas; text-align:center; color:#FF69B4">Introduction</h1>
    <h2 style = "font-family: Consolas; text-align:center">Why this Competition ❓</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    Australia's stunningly beautiful Great Barrier Reef is the world’s largest coral reef and home to 1,500 species of fish, 400 species of corals, 130 species of sharks, rays, and a massive variety of other sea life.Unfortunately, the reef is under threat, in part because of the overpopulation of one particular starfish – the coral-eating crown-of-thorns starfish (or COTS for short).
    </p>
    <h2 style = "font-family: Consolas; text-align:center">Goal of Competition 🥅</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    The goal of this competition is to accurately identify starfish in real-time by building an object detection model trained on underwater videos of coral reefs.
    </p>
</div>

<div style = "background: rgb(224,224,224);border-radius: 42px;">
    <h1 style = "font-family: Consolas; text-align:center; color:#FF69B4">Sponsors 💰</h1>
    <h2 style = "font-family: Consolas; text-align:center">TensorFlow</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    TensorFlow is a free and open-source software library for machine learning and artificial intelligence. It can be used across a range of tasks but has a particular focus on training and inference of deep neural networks.
    </p>
    <h2 style = "font-family: Consolas; text-align:center">CSIRO</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    The Commonwealth Scientific and Industrial Research Organisation is an Australian Government agency responsible for scientific research. CSIRO works with leading organisations around the world.
    </p>
    <h2 style = "font-family: Consolas; text-align:center">GBRF</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    The Great Barrier Reef Foundation is an Australian non-profit organisation established in 1999 to help protect and preserve the Great Barrier Reef. 
    </p>
</div>

<h2 style = "font-family: Consolas">More Details</h2>
<p style = "font-family : Lucida Sans Typewriter">Check <a href = "https://www.kaggle.com/c/petfinder-pawpularity-score/data">competition page</a> for details</p>
<h2 style = "font-family : Comic Sans MS">Let's dive in ⬇️</h2>

<center><img src = "https://img.shields.io/badge/Upvote-If%20you%20found%20this%20notebook%20useful-blue" width=400 height = 400></center>

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Install Libraries ⏬</h1>

In [ ]:
import torch
cuda_version_major = int(torch.version.cuda.split('.')[0])

!wget https://raw.githubusercontent.com/airctic/icevision/master/icevision_install.sh
!bash icevision_install.sh cuda11
!pip install torchtext==0.11.0 --upgrade

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Import Libraries 📚</h1>

In [ ]:
import os
import shutil

from kaggle_secrets import UserSecretsClient
import copy 
import time

import pandas as pd
from pandas_profiling import ProfileReport

import numpy as np
import ast

import cv2

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import torch
from icevision.all import *

import wandb

from colorama import Fore, Back, Style

from IPython.display import IFrame
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Initialize Constants 🔰</h1>

In [ ]:
TRAIN_PATH = "../input/tensorflow-great-barrier-reef/train_images"
HEIGHT, WIDTH = 720, 1280
image_size = 640

In [ ]:
def get_files(path):
    vid_ls = [os.path.join(path,f) for f in os.listdir(path)]
    return sorted(vid_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))

In [ ]:
vid_paths = [
    f'{TRAIN_PATH}/video_0',
    f'{TRAIN_PATH}/video_1',
    f'{TRAIN_PATH}/video_2',
]

files_ls = [get_files(vid_path) for vid_path in vid_paths]
len(files_ls)

train_df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")

In [ ]:
train_df

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; "> Data Preprocessing ⚒️🔬</h1>

In [ ]:
def get_oldpath(x):
    return os.path.join(vid_paths[x.video_id], f'{str(x.video_frame)}.jpg')

def get_newpath(x):
    filename = f"{x.video_id}_{x.video_frame}.jpg"
    return os.path.join("./dataset", filename)

def get_filename(x):
    return f"{x.video_id}_{x.video_frame}.jpg"

In [ ]:
# annotation 즉 레이블이 있는 것들만으로 추린다 (학습용 이니까)
train_df = train_df[train_df.annotations != "[]"]

# annotation 정보는 문자열이다 (예시."[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]")
# 이를 파이썬 객체로 변환하는 방법 중 하나로 ast.literal_eval 을 사용할 수 있다.
train_df["annotations"] = train_df["annotations"].map(lambda x : ast.literal_eval(x))

# DataFrame의 apply 는 레코드별로 지정된 함수를 적용한 뒤, 전체가 적용된 새로운 DataFrame을 반환한다.
# filepath는 실제 파일이 저장된 위치 (원본)
# newpath는 원본 파일을 복사해 옮겨놓을 위치 (복사본)
# filename은 단순히 파일이름
train_df["filepath"] = train_df.apply(lambda x : get_oldpath(x), axis=1)
train_df["newpath"] = train_df.apply(lambda x : get_newpath(x), axis=1)
train_df["filename"] = train_df.apply(lambda x : get_filename(x), axis=1)

os.makedirs("./dataset",exist_ok=True)

for i in tqdm(range(len(train_df))):
    src = train_df.iloc[i]["filepath"]
    dst = train_df.iloc[i]["newpath"]
    shutil.copy(src,dst)
    
train_df.head(2)

In [ ]:
train_df.iloc[0:2].explode('annotations').apply(lambda x : x.annotations["x"], axis=1)

In [ ]:
df = train_df

# DataFrame의 explode 메서드는 리스트를 풀어헤친다
# [{'x': 559, 'y': 213, 'width': 50, 'height': 32}] => {'x': 559, 'y': 213, 'width': 50, 'height': 32}
df = df.explode("annotations")

# 모든 이미지의 크기(높이 너비), 레이블이 모두 동일하다
# 엑셀에서 드래그로 주루룩 같은 갚을 채워넣는것 같은 일
df["width"] = [WIDTH]*len(df)
df["height"] = [HEIGHT]*len(df)
df["label"] = ["starfish"]*len(df)

# annotation에 담긴 x, y, width, height 대신, 
# 객체탐지에서 일반적으로 사용되는 xmin, ymin, xmax, ymax 용어와 알맞은 값으로 대체한다
df["xmin"] = df.apply(lambda x : x.annotations["x"], axis=1)
df["ymin"] = df.apply(lambda x : x.annotations["y"], axis=1)
df["xmax"] = df.apply(lambda x : x.annotations["x"]+x.annotations["width"], axis=1)
df["ymax"] = df.apply(lambda x : x.annotations["y"]+x.annotations["height"], axis=1)

# 바운딩박스의 우측 하단 좌표가 
# 이미지의 우측 하단 모서리를 벗어나는 경우
df.loc[df["xmax"] > 1280, "xmax"] = 1280
df.loc[df["ymax"] > 720, "ymax"] = 720

# 객체 탐지에 불필요한 열은 제거한다
df = df.drop(["video_id","sequence","video_frame","sequence_frame","image_id","annotations","filepath","newpath"], axis=1)
df = df.reset_index(drop=True)
df.head(3)

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Data Visualization 📊💹</h1>

<div style = "font-family : Lucida Sans Typewriter;background: rgb(224,224,224);border-radius: 25px;">
    <h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Weights and Biases</h2>
    <center><img src = "https://i.imgur.com/KISYcqD.png" width=200 height = 200></center>
    <a href = "https://wandb.ai/shanmukh/Protect%20Great%20Barrier%20Reef/runs/2eywhb67"; style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; text-align:center; font-size:25px">Checkout Dashboard created for this notebook</a>
    <p style = "font-family : Lucida Sans Typewriter">
    Weights and Biases is a set of Machine Learning tools used for experiment tracking, dataset versioning, and collaborating on ML projects. Weights and Biases is useful in many applications such as
    </p>  
    <ul>
          <li>Experiment Tracking</li>
          <li>Hyperparameter Tuning</li>
          <li>Data Visualization</li>
          <li>Data and model Versioning</li>
          <li>Collaborative Reports</li>
    </ul>
    <a href = "https://wandb.ai/site">Go to offocial website for more tutorials and Documentation</a>
</div>

In [ ]:
# Check https://www.kaggle.com/debarshichanda/pytorch-w-b-pawpularity-training for more details

# 이 섹션은 학습용 데이터셋을 W&B 대시보드에서 시각화하는 방법을 보여준다.
# 
# 1. W&B의 API 키를 발급 받는다
# 2. Add-ons => Secrets 메뉴를 통해, wandb_api 라는 키에 W&B API값을 할당한다
# 3. 아래 코드를 실행한다.

try:
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('''If you want to use your W&B account, Follow these steps :
            -> go to Add-ons {Below name of notebook} -> Secrets -> Add a new Secret
            -> Label = wandb_api
            -> Value = W&B access token from https://wandb.ai/authorize 
         ''')

In [ ]:
WANDB_CONFIG = {
   "project_name"    : "Protect Great Barrier Reef",
   "group_name"      : "IceVision Training",
   "job_type_data"   : "Data Visualization",
   "job_type_train"  : "Training",
   "anonymity"       : "must",
   "artifact"        : "training_data"
}

run = wandb.init(
    project = WANDB_CONFIG["project_name"],
    group = WANDB_CONFIG["group_name"],
    job_type = WANDB_CONFIG["job_type_data"],
    anonymous= WANDB_CONFIG["anonymity"]
)

In [ ]:
# 디스플레이될 테이블의 열을 정의한다
wb_table = wandb.Table(columns = [
    "filename", "Image", "xmin", "ymin", "xmax", "ymax"
])

# 학습용 데이터 중 처음 100개에 대해서 tqdm 한다
for i in tqdm(range(len(df))[:100]):
    row = df.loc[i]
    impath = os.path.join("./dataset",row["filename"])
    im = PIL.Image.open(impath)
    bbox = [{
            "position": {
                "minX": int(row['xmin']),
                "maxX": int(row["xmax"]),
                "minY": int(row["ymin"]),
                "maxY": int(row["ymax"])
            },
            "class_id": 1,
            "box_caption": "starfish",
            "domain": "pixel"
        }]
    
    # 이미지는 wandb.Image 객체 형태가 되어야한다.
    # 보다시피 PIL의 Image 형식을 그대로 사용할 수 있으며,
    # boxes 및 classes 라는 추가 파라미터를 설정하면 이미지내 객체 위치 및 범주를 결정해 보여줄 수 있다.
    image = wandb.Image(im,
                        boxes = {
                            "ground_truth": {
                                "box_data": bbox,
                                "class_labels" : {1: 'starfish'}
                            }
                        },
                        classes = [{"id": 0, "name": "background"}, {"id": 1, "name": "starfish"}]
                    )
    
    # 입력된 데이터가 준비되었으면,
    # 테이블의 행 데이터로 추가한다
    wb_table.add_data(row['filename'],
                      image,
                      row["xmin"],
                      row["ymin"],
                      row["xmax"],
                      row["ymax"]
                     )
    
wandb.log({'Data Visualization': wb_table})
run.finish()

In [ ]:
# 직접 W&B 대시보드에서 확인해도 되지만,
# 아래 명령을 수행하면 IFrame으로 W&B 대시보드 인터페이스를 주피터 노트북내 출력한다
frame = IFrame(run.url, width=1280, height=720)
frame

<div style = "font-family : Lucida Sans Typewriter;background: rgb(224,224,224);border-radius: 25px;">
    <h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Pandas Profiling</h2>
    <center><img src = "https://pandas-profiling.github.io/pandas-profiling/docs/assets/logo_header.png" width=200 height = 200></center>
    <p style = "font-family : Lucida Sans Typewriter">
    Pandas profiling is an open source Python module with which we can quickly do an exploratory data analysis with just a few lines of code. Besides, if this is not enough to convince us to use this tool, it also generates interactive reports in web format that can be presented to any person, even if they don’t know programming.
    </p>  
    <a href = "https://pandas-profiling.github.io/pandas-profiling/">Go to offocial website for documentation</a>
</div>

In [ ]:
train_report = ProfileReport(df,title="Metadata of Training images")
train_report.to_file("./train_metadata.html")
train_report

<div style = "font-family : Lucida Sans Typewriter;background: rgb(224,224,224);border-radius: 25px;">
    <h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">IceVision</h2>
    <center><img src = "https://airctic.com/0.11.0/images/icevision-logo-slogan.png" width=200 height = 200></center>
    <p style = "font-family : Lucida Sans Typewriter">
    IceVision is the first agnostic computer vision framework to offer a curated collection with hundreds of high-quality pre-trained models from torchvision, MMLabs, and soon Pytorch Image Models. It orchestrates the end-to-end deep learning workflow allowing to train networks with easy-to-use robust high-performance libraries such as Pytorch-Lightning and Fastai</p> 
    IceVision Unique Features:
    <ul>
        <li>Data curation/cleaning with auto-fix</li>
        <li>Access to an exploratory data analysis dashboard</li>
        <li>Pluggable transforms for better model generalization</li>
        <li>Access to hundreds of neural net models</li>
        <li>Acccss to multiple training loop libraries</li>
        <li>Multi-task training to efficiently combine object detection, segmentation, and classification models</li>
    </ul>
    <a href = "https://airctic.com/0.11.0/">Go to offocial website for documentation</a>
</div>

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Modelling 🏫</h1>

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Data Parser</h2>

How to create custom Parser for IceVision [[LINK](https://airctic.com/0.11.0/custom_parser/)]

데이터셋을 모델의 입력으로 만들기위한 클래스를 만든다. Pandas DataFrame의 내용을 파싱해야 하기 때문에, Parser 라고 불리는 객체를 활용하며, 객체 탐지에 사용되는 기본 객체는 ObjectDetectionRecord 이다.

In [ ]:
# 객체 탐지를 위한 레코드는 ObjectDetectionRecord 객체로 표현된다.
# Parser.generate_template 메서드는 레코드 객체를 만들기 위해 필요한 기본 템플릿을 출력한다.
template_record = ObjectDetectionRecord()
Parser.generate_template(template_record)

In [ ]:

# 보다시피 위에 출력된 MyParser 템플릿코드를 그대로 가져온 다음, 각 메서드를 구현하였다
class PGBRParser(Parser):
    def __init__(self, template_record, data_dir, df):
        super().__init__(template_record=template_record)

        self.data_dir = data_dir
        self.df = df
        self.class_map = ClassMap(list(self.df['label'].unique()))

    # 각 데이터 레코드(행)을 이터레이션하고, 
    # 각 이터레이션 마다 반환할 객체를 정의한다
    def __iter__(self) -> Any:
        for o in self.df.itertuples():
            yield o

    def __len__(self) -> int:
        return len(self.df)

    # 유니크 ID 할당에 사용될 수 있는 정보를 반환한다.
    # 여기서 o는 __iter__가 이터레이션마다 반환하는 객체를 의미한다
    def record_id(self, o) -> Hashable:
        return o.filename

    # 여기서 o는 __iter__가 이터레이션마다 반환하는 객체를 의미한다
    #
    # 보다시피 is_new 라는 인자가 존재한다. 사진 한 장에 여러 물체가 기록되어 있을 수 있다.
    # DataFrame 행을 하나씩 탐색하다가 동일한 이미지 이름을 가진 레코드(행)이 또 다시 발견된다면 이를 같은 레코드로 간주한다 (record_id에서 수행)
    # 그리고 record.detection.add_bboxes, record.detection.add_labels에 추가 bbox 및 레이블을 기록해 넣는다.
    # is_new는 지금까지 본 적 없는 record_id가 발견되었을 때만 수행한다.
    #
    # 이 내용으로 미루어 보면, 학습 데이터의 입력인 이미지는 set_filepath로 지정된다 (이미지 객체 자체가 아니다)
    def parse_fields(self, o, record, is_new):
        if is_new:
            record.set_filepath(os.path.join(self.data_dir,o.filename))
            record.set_img_size(ImgSize(width=o.width, height=o.height))
            record.detection.set_class_map(self.class_map)

        record.detection.add_bboxes([BBox.from_xyxy(o.xmin, o.ymin, o.xmax, o.ymax)])
        record.detection.add_labels([o.label])

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Data Loader</h2>

In [ ]:
# 데이터프레임에서 데이터를 파싱 (레이블과 BBOX)
template_record = ObjectDetectionRecord()
parser = PGBRParser(template_record, "./dataset", df)

train_records, valid_records = parser.parse()
print(parser.class_map)
# 여기까지하면 파싱이 완료된다.

train_tfms = tfms.A.Adapter([*tfms.A.aug_tfms(size=image_size, presize=512), tfms.A.Normalize()])
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(image_size), tfms.A.Normalize()])
# 여기까지하면 파싱된 데이터 적용할 변형들의 정의가 완료된다.

train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)
# 여기까지 하면 실제 모델에 입력될 Dataset 객체 생성이 완료된다.
# 보다시피 일부 데이터 증강까지 수행되었다. (presizing - https://notesbylex.com/presizing.html)

samples = [train_ds[0] for _ in range(3)]
show_samples(samples, ncols=3)
# 일부 데이터를 출력한다

In [ ]:
# model_type은 객체탐지용 전체 모델 구조를 선택한다
# - https://velog.io/@zeen263/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-3-Detection-2.-VFNet-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0
model_type = models.mmdet.vfnet

# 모델 구조 중 백본으로 사용할 모델을 선택한다
backbone = model_type.backbones.resnet50_fpn_mstrain_2x(pretrained=True)

# 그리고 이 둘을 결함해 전체 모델의 인스턴스를 생성한다
model = model_type.model(backbone=backbone, num_classes=len(parser.class_map))

# 모델에 입력될 데이터를 정의하는 DataLoader 객체를 생성한다
train_dl = model_type.train_dl(train_ds, batch_size=16, num_workers=4, shuffle=True)
valid_dl = model_type.valid_dl(valid_ds, batch_size=16, num_workers=4, shuffle=False)

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Metrics</h2>

In [ ]:
# VFNet이 타게팅한 데이터셋은 COCO 이다
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Training</h2>

In [ ]:
learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)

In [ ]:
# learn.lr_find()

In [ ]:
learn.fine_tune(30, 0.0003, freeze_epochs=5)

In [ ]:
torch.save(model.state_dict(), './gbr_vfnet_resnet50_fpn_mstrain_2x_30epochs.pt')

In [ ]:
!ls ../input/gbr-vfnet-resnet50-fpn-mstrain-2x-30epochspt/gbr_vfnet_resnet50_fpn_mstrain_2x_30epochs.pt

In [ ]:
state_dict = torch.load('../input/gbr-vfnet-resnet50-fpn-mstrain-2x-30epochspt/gbr_vfnet_resnet50_fpn_mstrain_2x_30epochs.pt')

In [ ]:
model_type = models.mmdet.vfnet
backbone = model_type.backbones.resnet50_fpn_mstrain_2x(pretrained=True)
model = model_type.model(backbone=backbone, num_classes=len(parser.class_map))

train_dl = model_type.train_dl(train_ds, batch_size=16, num_workers=4, shuffle=True)
valid_dl = model_type.valid_dl(valid_ds, batch_size=16, num_workers=4, shuffle=False)

model.load_state_dict(state_dict)

In [ ]:
learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)

In [ ]:
learn.fine_tune(1, 10e-5, freeze_epochs=0)

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Inference 🔮</h1>

In [ ]:
infer_dl = model_type.infer_dl([valid_ds[0],valid_ds[4],valid_ds[7],valid_ds[9]], batch_size=4, shuffle=False)
preds = model_type.predict_from_dl(model, infer_dl, keep_images=True)
show_preds(preds=preds)

In [ ]:
# shutil.make_archive('./checkpoints',
#                     'zip',
#                     './',
#                     'checkpoints')

# shutil.rmtree("./checkpoints")
# shutil.rmtree("./dataset")
# shutil.rmtree("./wandb")

<a href="./checkpoints.zip"> Download File </a>

<br>
<h3 style = "font-family: Consolas; text-align:center; color:#FF0000">If you come this far, you could've got some insights from this notebook. An upvote would be very helpful :). Kindly comment if there are any doubts or mistakes</h3>

<center><img src = "https://img.shields.io/badge/Completed-The%20End-brightgreen" width=300 height = 300></center>